In [10]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

### 1) Web scraping

In [11]:
#url = 'https://uk.indeed.com/jobs?q=software+developer&l=United+Kingdom&start=0'

In [12]:
def extract(page):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}
    url = f'https://uk.indeed.com/jobs?q=software+developer&l=United+Kingdom&start={page}'
    r = requests.get(url, headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [17]:
def transform(soup):
    divs = soup.find_all('div', class_ = 'jobsearch-SerpJobCard')
    for item in divs:
        title = item.find('a').text.strip()
        company = item.find('span', class_ = 'company').text.strip()
        try:
            salary = item.find('span', class_ = 'salaryText').text.strip()
        except:
            salary = ''
        summary = item.find('div', class_ = 'summary').text.strip().replace('\n', '')
        location = item.find('div', class_ = 'recJobLoc').get('data-rc-loc')
        post_date = item.find('span', class_ = 'date').text.strip()
        today = datetime.today().strftime('%Y-%m-%d')
        job_url = 'https://indeed.com' + item.find('a').get('href')
        
        
        job = {
            'title' : title,
            'company' : company,
            'salary' : salary,
            'summary' : summary,
            'location' : location,
            'post_date' : post_date,
            'today' : today,
            'job_url' : job_url
        }
        joblist.append(job)
    return

joblist = []

for i in range(0, 991, 10):
    c = extract(0)        
    transform(c)

### 2) DataFrame

In [19]:
df = pd.DataFrame(joblist)

df.head()

df.to_csv('jobs_scraped.csv')